In [6]:
from pyplasm import *
from appartamento import *
from tetto import *

def stairs(dx,dy,dz):
    k = 0
    n = dy/.25
    n -= n%1
    treadStep = dy/n
    riserStep = dz/n
    firstStep = CUBOID([dx,treadStep,riserStep])
    step = MKPOL([[[0,0],[0,riserStep*2],[treadStep,riserStep*2],[treadStep,riserStep]],[[1,2,3,4]],1])
    step = PROD([QUOTE([dx]), step])
    stairs = [firstStep, T(2)(treadStep), step] 
    for c in range(2,int(n)):
        stairs.append(T([2,3])([treadStep, riserStep]))
        stairs.append(step)
    stairs.append(T([2,3])([treadStep, riserStep]))
    #stairs.append(CUBOID([2*dx + .25, 1., riserStep]))
    return stairs

def pianerottolo(dx, dy, dz, a, nApp):
    n = dy/.25
    n -= n%1
    riserStep = dz/n
    if a:
        app1 = appartamento(1)[0]
        #b1 = PROD([BOX([1,2])(app1), QUOTE([riserStep])])
        #app1 = STRUCT([b1, T(3)(riserStep), app1])
        app = appartamento(3)[0]
        #b = PROD([BOX([1,2])(app), QUOTE([riserStep])])
        #app = STRUCT([b, T(3)(riserStep), app])
        mx2 = appartamento(2)[1]
        app2 = appartamento(2)[0]
        #b2 = PROD([BOX([1,2])(app2), QUOTE([riserStep])])
        #app2 = STRUCT([b2, T(3)(riserStep), app2])
        piano = T(2)(-dy/2.5)(CUBOID([dx, dy/1.1, riserStep]))
        piano = STRUCT([piano, app])
        if nApp > 1: piano = STRUCT([piano, T(1)(dx), app1])
        if nApp > 2:
            if nApp >6: lista = [T(1)(-dx+mx2/1.2)]
            else: lista = [T(1)(-dx+mx2/2)]
            for r in range(2,nApp):
                lista += [app2, T(1)(mx2-.7)]
            appartamenti = STRUCT(lista)
            nAPP = nApp
            if nApp == 3:
                nAPP = 0
            piano = STRUCT([piano, T([1,2])([dx-(nApp-2)*mx2/(5+nAPP), dy/2.05]), appartamenti])
    else:
        piano = CUBOID([dy-.1, dx, riserStep])
    return piano

def pianoScale(dx, dy, dz, nApp):
    rampa = STRUCT(stairs(dx, dy, dz))
    rampa1 = stairs(dx, dy, dz)
    rampa2 = STRUCT([S(2)(-1), rampa])
    rampa1.append(pianerottolo(dx, dy, dz, 0, nApp))
    rampa1 = STRUCT(rampa1)
    muro = MKPOL ([[[0., 0.], [dx*2.45, 0.], [0., dy*1.4], [dx*2.45, dy*1.4]],[[1,3],[3,4],[2,4]],[[1]]])
    muro = OFFSET([0, .1, dz * 2])(muro)
    muro = OFFSET([.1, 0,0])(muro)
    if nApp > 2: 
        mx = appartamento(2)[1]
        n = (nApp - 2)/2
        m = 0
    else: 
        n = 0
        mx = 0
        m = appartamento(2)[1]
    rampa = STRUCT([rampa1, T(1)(-.1), muro, T(1)(.1), T([1,2,3])([dx + .25, dy, dz - .17]), rampa2])
    return [rampa, T([1,2])([-dx + mx*n + m/2, dy])]


In [7]:
def piano(dx, dy, dz, nApp):
    n = dy/.25
    n -= n%1
    riserStep = dz/n
    p = []
    p.append(T([1,2,3])([-2.5*dx, 0, 2*dz-.34]))
    p.append(S(2)(-1))
    if nApp > 2: 
        mx = appartamento(2)[1]-1
        m = 0
    else: 
        mx = 0
        m = appartamento(2)[1]
    p.append(pianerottolo(m + (nApp - 2)*(mx)+.3, dy * 2.5 , dz*2.5, 1, nApp))
    return p

In [8]:
def rampaScale(dx, dy, dz, nApp, ultimo):
    if nApp > 2: 
        mx = appartamento(2)[1]
        m = 0
    else: 
        mx = 0
        m = appartamento(2)[1]
    if not ultimo:
        scale = pianoScale(dx, dy, dz, nApp)[0]
        scale = STRUCT([pianoScale(dx,dy,dz,nApp)[1], scale])
        floor = piano (dx, dy, dz, nApp)
        scale = STRUCT([S(2)(-1), scale, floor[1], floor[2]])
    else:
        floor = piano (dx, dy, dz, nApp)
        scale = STRUCT([S(2)(-1), floor[1], floor[2]])
    return scale

def numeroPiani(h, dx, dy, dz, nApp):
    n = dy/.25
    n -= n%1
    riserStep = dz/n
    rampa = rampaScale(dx, dy, dz, nApp, 0)
    ultimoPiano = rampaScale(dx, dy, dz, nApp, 1)
    finale = [rampa]
    my = appartamento(2)[1]-1
    if nApp > 2: 
        mx = appartamento(2)[1]-1
        m = 0
    else: 
        mx = 0
        m = appartamento(2)[1]
    for r in range(1, h):
        finale.append(T(3)(2*(dz-riserStep)))
        if r == h-1: finale.append(ultimoPiano)
        else: finale.append(rampa)
    finale.append(T([2,3])([-2,2*(dz-riserStep)]))
    finale.append(T(2)(-dy/2.5)(CUBOID([m + (nApp - 2)*(mx)+.3, my, riserStep])))
    rampaFinale = STRUCT(finale)
    return rampaFinale

In [9]:
def disegnaTetto():
    ax = appartamento(2)[1]
    ay = appartamento(2)[2]
    p1 = [.0, .0, .0]
    p2 = [ax+2*ay, 0., 0.]
    p3 = [0., ax, .0]
    p4 = [ax+2*ay, ax, .0]
    p5 = [ax/4., 0, 2.]
    p6 = [0, ay/4., 2.]
    p7 = [2*ay+(3./4)*ax, 0., 2.]
    p8 = [0,(3./4)*ax, 2.]
    piano1 = trovaPiano(p1, p2, p6)
    piano2 = trovaPiano(p1, p3, p5)
    piano3 = trovaPiano(p2, p4, p7)
    piano4 = trovaPiano(p3, p4, p8)
    i1 = intersezioniPiani(piano1, piano3, 2.)
    i2 = intersezioniPiani(piano3, piano4, 2.)
    i3 = intersezioniPiani(piano4, piano2, 2.)
    i4 = intersezioniPiani(piano2, piano1, 2.)
    i = [i1, i2, i3, i4]
    pti = []
    tettoia = []
    for el in i:
        tettoia.append(el[1])
        for p in el:
            pti.append(p)

    pol = [[1,2,3,4],[3,4,5,6],[5,6,7,8],[7,8,1,2]]
    d = MKPOL([pti, pol, [[1]]])
    d = TEXTURE("roof.jpg")(d)

    rette = []
    for r in range (0, len(tettoia)):
        tmp = []
        if r == len(tettoia) - 1:
            tmp.append(retta(tettoia[r], tettoia[0]))
            tmp.append([tettoia[r][0], tettoia[r][1]])
            tmp.append([tettoia[0][0], tettoia[0][1]])
        else:
            tmp.append(retta(tettoia[r], tettoia[r+1]))
            tmp.append([tettoia[r][0], tettoia[r][1]])
            tmp.append([tettoia[r+1][0], tettoia[r+1][1]])
        rette.append(tmp)

    ordinate = ordinaRette(rette)
    t = disegnaPiano(ordinate)
    tot = STRUCT([d, T(3)(2.), t])
    return tot

In [10]:
#m + (nApp - 2)*(mx)+.3, dy * 2.5 , dz*2.5, 1, nApp
dx = 1.
dy = 2.5
dz = 1.7
n = dy/.25
n -= n%1
riserStep = dz/n

tetto = disegnaTetto()

casa = numeroPiani(3, dx, dy, dz, 2)
#casa = S([1,2])(.75)(casa)
box = BOX([1,2])(casa)
casa = STRUCT([casa, T([1,2,3])([-appartamento(2)[2],-appartamento(2)[1]/3.+.3,6*(dz-riserStep)]),tetto])
VIEW(casa)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000050D8B40> >